In [1]:
pip install qeds fiona geopandas xgboost gensim folium pyLDAvis descartes statspy 

     |████████████████████████████████| 14.8MB 241kB/s 
     |████████████████████████████████| 972kB 44.8MB/s 
     |████████████████████████████████| 1.6MB 42.0MB/s 
     |████████████████████████████████| 235kB 47.1MB/s 
     |████████████████████████████████| 10.9MB 5.2MB/s 
  Created wheel for qeds: filename=qeds-0.6.2-cp36-none-any.whl size=27821 sha256=8626d1cd26972cdc7ae251e3072121f9b6ca7de0fcc73066ca3e9042698cf1e5
  Stored in directory: /root/.cache/pip/wheels/b7/0b/74/c09109813c2b6116a2d4f2833c354b24163672f846a50fc7b4
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=dbabb3ef68fc5d07761330a10b55e36da71fba9fdb86beb3335756a8038444aa
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for statspy: filename=statspy-0.1.0a1-cp36-none-any.whl size=26104 sha256=13b4a195a346bd99e03d59e7e8609d886ac81e21da3c3e3aa4a9234d91157104
  Stored in directory: /root/.cache/pip/wheels

In [2]:
#Import
import pandas as pd
import numpy as np
import patsy
import statspy as sp
from sklearn import linear_model, ensemble, base, neural_network
#
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#Load file - Demeaned by Location
from google.colab import drive
drive.mount('/content/drive')

df_path="/content/drive/My Drive/ML/KCYPS2010-m1e4e1_new.dta"
df=pd.read_stata(df_path)
df.describe()

Mounted at /content/drive


,id,year,panel,gender,age,school_loc,ind_loc,school,grade,coeduw,f_edu,m_edu,health,exercise_time,ego_resilience1,ego_resilience2,ego_resilience3,ego_resilience4,ego_resilience5,ego_resilience6,ego_resilience7,ego_resilience8,ego_resilience9,ego_resilience10,ego_resilience11,ego_resilience12,ego_resilience13,ego_resilience14,satisfaction1,satisfaction2,satisfaction3,homework1_time1,homework1_time2,homework1_time3,homework1_time4,homework2_time1,homework2_time2,homework2_time3,homework2_time4,homework3_time1,...,temp_min_n,temp_min,temp_max_n,temp_max,rainfall_n,rainfall,wind_max_n,wind_max,wind_avg_n,wind_avg,m_study,f_study,female,exercise_time1,loc_yr,hwtime_schday,hwtime_nonschday,withfriend_time_sch,withfriend_time_nsch,satisfaction,aggressive,symptom_phy,depression,ego_resilience,social_cohesion,move,income,aggressive_p1,depression_p1,symptom_phy_p1,ego_resilience_p1,aggressive_p2,depression_p2,symptom_phy_p2,ego_resilience_p2,satisfaction_p3,aggressive_p3,depression_p3,symptom_phy_p3,ego_resilience_p3
count,25468.000000,25468.000000,25468.000000,25013.000000,25093.000000,19490.000000,25468.000000,25010.000000,25033.000000,13073.000000,24328.000000,24536.000000,25004.000000,25008.000000,13182.000000,13181.000000,13182.000000,13182.000000,13181.000000,13182.000000,13182.000000,13182.000000,13182.000000,13181.000000,13181.000000,13181.000000,13181.000000,13180.000000,23472.000000,23474.000000,23474.000000,24832.000000,24832.000000,24880.000000,24880.000000,24778.000000,24778.000000,24802.000000,24802.000000,24884.000000,...,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,25468.000000,25468.000000,25468.000000,25008.000000,25468.000000,24607.000000,24686.000000,24866.000000,24891.000000,23569.000000,16308.000000,16308.000000,16308.000000,13460.000000,24993.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000
mean,87913.105623,2012.853760,2.117952,1.467397,13.397601,2741.557414,2746.313570,1.862015,2.972636,2.451771,2.884865,2.698769,3.281715,2.913988,3.074040,3.036871,3.082385,3.043013,2.948866,3.035958,2.682749,2.949325,2.829237,3.037933,2.803657,2.780821,2.958197,3.173976,3.287960,2.791088,3.238434,0.427150,12.297922,0.427653,8.462379,0.483857,8.254702,0.473430,5.805338,0.847452,...,7.308627,3.960305,16.920568,13.723699,3.196835,1.948080,7.116713,7.314519,1.811941,1.880025,13.040051,13.441613,0.459047,1.913988,272.760803,2.211384,2.227036,0.841032,2.089274,0.001044,-0.000357,-0.000072,-0.001104,0.001772,0.000948,0.072444,4.713395,0.000290,-0.000158,0.000303,0.000600,0.000287,-0.000162,0.000298,0.000604,0.000726,0.000284,-0.000162,0.000298,0.000603
std,50006.514034,1.921218,0.757773,0.498946,2.308522,894.825103,895.389902,0.762092,1.736361,0.780727,1.789775,1.580844,0.593584,1.264765,0.650978,0.696299,0.771551,0.659711,0.868586,0.752759,0.917145,0.789322,0.789537,0.768452,0.811332,0.810767,0.776063,0.686779,0.682429,0.883672,0.724015,0.655464,14.657259,0.743685,13.559300,0.749488,13.700982,0.851854,12.110255,1.186044,...,3.089340,3.432631,2.439558,2.750279,4.701747,3.439408,2.055955,2.171004,0.962986,1.027465,3.835263,4.485485,0.498330,1.264866,147.699905,1.630425,2.155110,1.047577,2.007680,0.890658,0.890302,0.914353,0.940720,0.926650,0.890123,0.259226,2.632188,0.047453,0.058434,0.045313,0.070305,0.047391,0.058327,0.045203,0.070110,0.055151,0.047342,0.058376,0.045279,0.070275
min,14201.000000,2010.000000,1.000000,1.000000,9.000000,-9.000000,1001.000000,1.000000,1.000000,1.000000,-9.000000,-9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.983333,-5.644444,10.175000,5.533333,0.000000,

In [4]:
## (Outcome1) satisfaction1

#satisfaction, health, aggressive, depression

#Set dataset/cleaning
formula="""
satisfaction1 ~ pm10_bad + income + age + p_study + temp_avg + rainfall + wind_avg
"""

df["p_study"] = ((df["m_study"] >=12)|(df["f_study"] >=12)).astype(int)

satisfaction1, X = patsy.dmatrices(formula, df, return_type="dataframe")
# some categories are empty after dropping rows will Null, drop now
#X = X.loc[:, X.sum() > 0]
satisfaction1 = satisfaction1.iloc[:, 0]
treatment_variable = "pm10_bad"
treatment = X["pm10_bad"]
Xl = X.drop(["Intercept","pm10_bad"], axis=1)
loc_id = df.loc[X.index, "ind_loc"].astype("category")

import re
# remove [ ] from names for compatibility with xgboost
Xl = Xl.rename(columns=lambda x: re.sub('\[|\]','_',x))

In [5]:
################################################################################

##Step 1: Best ML 
# Def of SE
def get_treatment_se(fit, cluster_id, rows=None):
    if cluster_id is not None:
        if rows is None:
            rows = [True] * len(cluster_id)
        vcov = sm.stats.sandwich_covariance.cov_cluster(fit, cluster_id.loc[rows])
        return np.sqrt(np.diag(vcov))

    return fit.HC0_se

# Def of Generic ML model
def generic_ml_model(x, y, treatment, model, n_split=100, n_group=5, cluster_id=None):
    nobs = x.shape[0]

    blp = np.zeros((n_split, 2))
    blp_se = blp.copy()
    blp_ci_l = blp.copy()
    blp_ci_u = blp.copy()
    blp_pvalue= np.zeros((n_split, 4))

    gate = np.zeros((n_split, n_group))
    affected = np.zeros((n_split, 3))    
    gate_se = affected.copy()
    affected_ci_l = affected.copy()
    affected_ci_u = affected.copy()
    affected_pvalue = np.zeros((n_split, 6))

    blp_new = np.zeros((1, 2))
    affected_new = np.zeros((1, 3))

    baseline = np.zeros((nobs, n_split))
    cate = baseline.copy()
    lamb = np.zeros((n_split, 2))

    for i in range(n_split):
        main = np.random.rand(nobs) > 0.5
        rows1 = ~main & (treatment == 1)
        rows0 = ~main & (treatment == 0)

        mod1 = base.clone(model).fit(x.loc[rows1, :], (y.loc[rows1]))
        mod0 = base.clone(model).fit(x.loc[rows0, :], (y.loc[rows0]))

        B = mod0.predict(x)
        S = mod1.predict(x) - B

        if B.var()==0:
          B = B + np.random.normal(0, 0.1, nobs)
        if S.var()==0:
          S = S + np.random.normal(0, 0.1, nobs)

        baseline[:, i] = B
        cate[:, i] = S
        ES = S.mean()

        #Creating weights
        p = treatment.mean()
        num=main[main==1].shape[0]

        weight=np.repeat((1/(p*(1-p))), num)

################################################################################

        ## BLP
        # assume P(treat|x) = P(treat) = mean(treat)
        p = treatment.mean()
        reg_df = pd.DataFrame(dict(
            y=y, B=B, treatment=treatment, S=S, main=main, excess_S=S-ES
        ))
        reg =smf.wls("y ~ B + S + I(treatment-p) + I((treatment-p)*(S-ES))", data=reg_df.loc[main, :], weights = weight)
        #reg =smf.ols("y ~ B + I(treatment-p) + I((treatment-p)*(S-ES))", data=reg_df.loc[main, :]);
        reg_fit = reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main] })

        blp_se[i, :] = get_treatment_se(reg_fit, cluster_id, main)[3:]
        
        #beta1 BLP
        beta1_blp = reg_fit.params.iloc[3]
        beta1_blp_ci_l = reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,0][3]
        beta1_blp_ci_u= reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,1][3]

        #beta2 BLP
        beta2_blp = reg_fit.params.iloc[4]
        beta2_blp_ci_l = reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,0][4]
        beta2_blp_ci_u= reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,1][4]

        #beta1, beta2
        blp[i,:]=beta1_blp, beta2_blp

        #BLP pvalue
        blp1_pvalue=reg_fit.pvalues[2]
        blp2_pvalue=reg_fit.pvalues[3]

        blp1_1=(beta1_blp<0)*1*(blp1_pvalue/2) + (beta1_blp>0)*1*(1-blp1_pvalue/2)
        blp1_2=(beta1_blp<0)*1*(1-blp1_pvalue/2) + (beta1_blp>0)*1*(blp1_pvalue/2)
        #blp1=min(1, 4*min(blp1_1, blp1_2))

        blp2_1=(beta2_blp<0)*1*(blp2_pvalue/2) + (beta2_blp>0)*1*(1-blp2_pvalue/2)
        blp2_2=(beta2_blp<0)*1*(1-blp2_pvalue/2) + (beta2_blp>0)*1*(blp2_pvalue/2)
        #blp2=min(1, 4*min(blp2_1, blp2_2))

        blp_pvalue[i,:] = blp1_1, blp1_2, blp2_1, blp2_2

        blp_ci_l[i,:]=beta1_blp_ci_l, beta2_blp_ci_l
        blp_ci_u[i,:]=beta1_blp_ci_u, beta2_blp_ci_u

        lamb[i, 0] = reg_fit.params.iloc[-1]**2 * S.var()

################################################################################

        ## GATEs
        cutoffs = np.quantile(S, np.linspace(0,1, n_group + 1))
        cutoffs[-1] += 1
        for k in range(n_group):
            reg_df[f"G{k}"] = (cutoffs[k] <= S) & (S < cutoffs[k+1])

        g_form = "y ~ B + S + " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        g_reg = smf.wls(g_form, data=reg_df.loc[main, :], weights = weight)
        #g_reg = smf.ols(g_form, data=reg_df.loc[main, :])
        g_fit = g_reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main]})
        gate[i, :] = g_fit.params.values[3:] #g_fit.params.filter(regex="G").values

        lamb[i, 1] = (gate[i,:]**2).sum()/n_group

################################################################################

        ## Most/Least affected
        g_form = "y ~ B + S +  " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        g_reg = smf.wls(g_form, data=reg_df.loc[main, :], weights = weight)
        g_fit = g_reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main]})



        #Most affected 
        most_coef=g_fit.params[6]        
        most_pvalue=g_fit.pvalues[6]
        most_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[6,0]
        most_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[6,1]


        #Least affected 
        least_coef=g_fit.params[2]
        least_pvalue=g_fit.pvalues[2]
        least_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[2,0]
        least_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[2,1]

        #Difference
        test=g_fit.t_test('I((treatment - p) * G4)-I((treatment - p) * G0)=0')
        dir(test)
        diff_coef= test.effect
        #diff_coef=most_coef-least_coef
        diff_pvalue=test.pvalue
        diff_ci_l=test.conf_int(alpha=0.05)[0][0]
        diff_ci_u=test.conf_int(alpha=0.05)[0][1]

        #pvalue
        most_p1=(most_coef<0)*1*(most_pvalue/2)+(most_coef>0)*1*(1-most_pvalue/2)
        most_p2=(most_coef<0)*1*(1-most_pvalue/2)+(most_coef>0)*1*(most_pvalue/2)
        least_p1=(least_coef<0)*1*(least_pvalue/2)+(least_coef>0)*1*(1-least_pvalue/2)
        least_p2=(least_coef<0)*1*(1-least_pvalue/2)+(least_coef>0)*1*(least_pvalue/2)
        diff_p1=(diff_coef<0)*1*(diff_pvalue/2)+(diff_coef>0)*1*(1-diff_pvalue/2)
        diff_p2=(diff_coef<0)*1*(1-diff_pvalue/2)+(diff_coef>0)*1*(diff_pvalue/2)

        affected[i,:]=most_coef,least_coef,diff_coef
        affected_ci_l[i,:]=most_ci_l,least_ci_l,diff_ci_l
        affected_ci_u[i,:]=most_ci_u,least_ci_u,diff_ci_u
        affected_pvalue[i,:]=most_p1, most_p2, least_p1, least_p2, diff_p1, diff_p2


################################################################################
    blp_pvalue= np.nanmedian(blp_pvalue, axis=0)
    affected_pvalue= np.nanmedian(affected_pvalue, axis=0)

    blp_new[0][0] = min(1, 4*min(blp_pvalue[0], blp_pvalue[1]))
    blp_new[0][1] = min(1, 4*min(blp_pvalue[2], blp_pvalue[3]))
    blp_pvalue = blp_new

    affected_new[0][0] = min(1, 4*min(affected_pvalue[0], affected_pvalue[1]))
    affected_new[0][1] = min(1, 4*min(affected_pvalue[2], affected_pvalue[3]))
    affected_new[0][2] = min(1, 4*min(affected_pvalue[4], affected_pvalue[5]))
    affected_pvalue = affected_new


    out = dict(
        blp = blp,
        blp_ci_l=blp_ci_l, 
        blp_ci_u=blp_ci_u,
        blp_pvalue=blp_pvalue, 
        gate=gate,
        gate_se=gate_se, 
        blp_se=blp_se,
        affected = affected,
        affected_ci_l =affected_ci_l, 
        affected_ci_u= affected_ci_u, 
        affected_pvalue = affected_pvalue,
        Lambda=lamb, 
        baseline=baseline, 
        cate=cate,
        name=type(model).__name__
    )
    return out

In [6]:
#Step 1: Tune ML hyperparameters: Best: Boosting /random forest
import xgboost as xgb
models = [
    xgb.XGBRegressor(n_estimators=100, max_depth=3, reg_lambda=2.0, reg_alpha=0.0, objective="reg:squarederror"),
    ensemble.RandomForestRegressor(n_estimators=100, min_samples_leaf=20)
]
#models = [
#    linear_model.ElasticNetCV(cv=5, n_alphas=25, max_iter=1000, tol=1e-3, n_jobs=-1, l1_ratio=0.95),
#    ensemble.RandomForestRegressor(n_estimators=100, min_samples_leaf=20),
#    xgb.XGBRegressor(n_estimators=100, max_depth=3, reg_lambda=2.0, reg_alpha=0.0, objective="reg:squarederror"),
#    neural_network.MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, activation="relu", solver="adam", tol=1e-3, early_stopping=True, alpha=0.0001, learning_rate="constant", shuffle=True)
#]

#Set parameters
x = Xl
treatment=treatment
n_split=100
cluster_id=loc_id

#Tune parameters
y=satisfaction1
n_group=5

In [7]:

kw = dict(x=x, treatment= treatment, n_split=n_split, n_group=n_group, cluster_id=cluster_id)
@ignore_warnings(category=ConvergenceWarning)


def evaluate_models(models, y, **other_kw):
    all_kw = kw.copy()
    all_kw["y"] = y
    all_kw.update(other_kw)
    return list(map(lambda x: generic_ml_model(model=x, **all_kw), models))

In [8]:
results = evaluate_models(models=models, y=y)

In [9]:
#Create SES variable
#df["p_study"] = ((df["m_study"] >=12)|(df["f_study"] >=12)).astype(int)

#Set variables: 2
df2 = df.loc[X.index, :]
variables = [
    "income", "p_study","age"
    ]

In [10]:
###THIS
## CLAN: Most & Least Affected ********************

#Create SES variable
#df["p_study"] = ((df["m_study"] >=12)|(df["f_study"] >=12)).astype(int)

def cov_mean_by_group(y, res, cluster_id):
    #n_group = 3
    nob=res["gate"].shape[0]
    gate = np.zeros((nob,3))
    gate_se = gate.copy()
    gate_ci_l=gate.copy()
    gate_ci_u=gate.copy()
    affected_pvalue = np.zeros((nob,6))
    affected_new=np.zeros((1,3))
    dat = y.to_frame()
    thres=0.2

    for i in range(res["cate"].shape[1]):
        S = res["cate"][:, i]
        high_effect=np.quantile(S, 1-thres)
        low_effect=np.quantile(S, thres)
        h = (S > high_effect)*1
        l = (S < low_effect)*1

        if h.var()==0:
          h=(np.random.uniform(len(h))<0.1)*1
        if l.var()==0:
          l=(np.random.uniform(len(l))<0.1)*1

        dat["h"] =h
        dat["l"] =l

        main= (dat["h"]==1) | (dat["l"]==1)


        ## Most/Least affected
        reg_df = pd.DataFrame(dict(
            y=y, h=h, l=l, main=main
        ))
        g_form = "y ~ -1 + h + l"
        g_reg = smf.ols(g_form, data=reg_df.loc[main,:])
        g_fit = g_reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main]})

        #Most affected 
        most_coef=g_fit.params[0]        
        most_pvalue=g_fit.pvalues[0]
        most_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[0,0]
        most_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[0,1]


        #Least affected 
        least_coef=g_fit.params[1]
        least_pvalue=g_fit.pvalues[1]
        least_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[1,0]
        least_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[1,1]

        #Difference
        test=g_fit.t_test('h-l=0')
        dir(test)
        diff_coef=most_coef-least_coef
        diff_pvalue=test.pvalue
        diff_ci_l=test.conf_int(alpha=0.05)[0][0]
        diff_ci_u=test.conf_int(alpha=0.05)[0][1]

        #pvalue
        most_p1=(most_coef<0)*1*(most_pvalue/2)+(most_coef>0)*1*(1-most_pvalue/2)
        most_p2=(most_coef<0)*1*(1-most_pvalue/2)+(most_coef>0)*1*(most_pvalue/2)
        least_p1=(least_coef<0)*1*(least_pvalue/2)+(least_coef>0)*1*(1-least_pvalue/2)
        least_p2=(least_coef<0)*1*(1-least_pvalue/2)+(least_coef>0)*1*(least_pvalue/2)
        diff_p1=(diff_coef<0)*1*(diff_pvalue/2)+(diff_coef>0)*1*(1-diff_pvalue/2)
        diff_p2=(diff_coef<0)*1*(1-diff_pvalue/2)+(diff_coef>0)*1*(diff_pvalue/2)

        gate[i,:]=most_coef,least_coef,diff_coef
        gate_ci_l[i,:]=most_ci_l, least_ci_l, diff_ci_l
        gate_ci_u[i,:]=most_ci_u, least_ci_u, diff_ci_u
        affected_pvalue[i,:]=most_p1, most_p2, least_p1, least_p2, diff_p1, diff_p2
#####
    affected_pvalue=np.nanmedian(affected_pvalue, axis=0)

    affected_new[0][0] = min(1, 4*min(affected_pvalue[0], affected_pvalue[1]))
    affected_new[0][1] = min(1, 4*min(affected_pvalue[2], affected_pvalue[3]))
    affected_new[0][2] = min(1, 4*min(affected_pvalue[4], affected_pvalue[5]))
    affected_pvalue = affected_new.ravel()


    out = pd.DataFrame(dict(
        mean=np.nanmedian(gate, axis=0),
        se=np.nanmedian(gate_se, axis=0),
        affected_pvalue=affected_pvalue,
        ci_l=np.nanmedian(gate_ci_l,axis=0), 
        ci_u=np.nanmedian(gate_ci_u,axis=0),
        group=["20% Most","20% Least", "Difference"]
    ))

    return out


In [11]:
cov_mean_by_group(df2["income"], results[0], cluster_id)

,mean,se,affected_pvalue,ci_l,ci_u,group
0,4.436827,0.0,2.055968e-288,4.208796,4.674839,20% Most
1,4.914090,0.0,1.809532e-291,4.654538,5.167426,20% Least
2,-0.487965,0.0,4.470142e-03,-0.783624,-0.185037,Difference


In [12]:
def compute_group_means_for_results(results):
    to_cat = []
    for res in results:
        for v in variables:
            to_cat.append(
                cov_mean_by_group(df2[v], res, loc_id)
                .assign(method=res["name"], variable=v)
            )

    group_means = pd.concat(to_cat, ignore_index=True)
    #group_means["plus2sd"] = group_means.eval("mean + 1.96*se")
    #group_means["minus2sd"] = group_means.eval("mean - 1.96*se")
    return group_means

#######
group_means = compute_group_means_for_results(results)
#######
#Rename Method Name
nob=group_means.shape[0]
for i in range(nob):
  if group_means["method"][i]=='ElasticNetCV':
    group_means["method"][i]='Elastic Net'
  elif group_means["method"][i]=='MLPRegressor':
    group_means["method"][i]='Neural Network'
  elif group_means["method"][i]=='RandomForestRegressor':
    group_means["method"][i]='Random Forest'
  else:
    group_means["method"][i]='Booster'

#Rename variable name
for i in range(nob):
  if group_means["variable"][i]=='income':
    group_means["variable"][i]='Income'
  elif group_means["variable"][i]=='age':
    group_means["variable"][i]='Age'
  else:
    group_means["variable"][i]='SES'

#Rename columns
group_means.rename(columns={'ci_u':'Upper CI','ci_l':'Lower CI', 'affected_pvalue':'pvalue'}, inplace=True)


group_means=group_means[["mean", "pvalue", "Lower CI", "Upper CI","method","variable"]]
print(group_means)

#Save CLAN to csv
group_means.to_csv("/content/drive/My Drive/ML/new2/Stage3_CLAN_satisfaction1.csv", index=False, header=True)

#for Table
clan_path="/content/drive/My Drive/ML/new2/Stage3_CLAN_satisfaction1.csv"
clan=pd.read_csv(clan_path)
clan=pd.DataFrame(dict(mean=clan["mean"],se=clan["pvalue"],lower=clan["Lower CI"], upper=clan["Upper CI"]))
print(clan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to b

         mean         pvalue   Lower CI   Upper CI         method variable
0    4.436827  2.055968e-288   4.208796   4.674839        Booster   Income
1    4.914090  1.809532e-291   4.654538   5.167426        Booster   Income
2   -0.487965   4.470142e-03  -0.783624  -0.185037        Booster   Income
3    0.950191   0.000000e+00   0.937999   0.962556        Booster      SES
4    0.961534   0.000000e+00   0.949834   0.972967        Booster      SES
5   -0.010569   4.583956e-01  -0.027811   0.006554        Booster      SES
6   14.202703   0.000000e+00  13.870509  14.525090        Booster      Age
7   12.491006   0.000000e+00  12.324847  12.663485        Booster      Age
8    1.658204   9.010222e-17   1.288454   2.062461        Booster      Age
9    4.493447  1.960358e-284   4.243432   4.748002  Random Forest   Income
10   4.826090   0.000000e+00   4.606462   5.067269  Random Forest   Income
11  -0.337240   7.831358e-02  -0.639690  -0.014337  Random Forest   Income
12   0.963165   0.000000e

In [13]:
#Step 3 : Create table
clan_path="/content/drive/My Drive/ML/new2/Stage3_CLAN_satisfaction1.csv"
clan=pd.read_csv(clan_path)


#Create Dataframe
c = np.arange(54).reshape(9,6)
c_df=pd.DataFrame(c)

#1st row: mean
#income
c_df.iloc[0,0]=round(clan.iloc[0,0],3)
c_df.iloc[0,1]=round(clan.iloc[1,0],3)
c_df.iloc[0,2]=round(clan.iloc[2,0],3)
c_df.iloc[0,3]=round(clan.iloc[9,0],3)
c_df.iloc[0,4]=round(clan.iloc[10,0],3)
c_df.iloc[0,5]=round(clan.iloc[11,0],3)
#SES
c_df.iloc[3,0]=round(clan.iloc[3,0],3)
c_df.iloc[3,1]=round(clan.iloc[4,0],3)
c_df.iloc[3,2]=round(clan.iloc[5,0],3)
c_df.iloc[3,3]=round(clan.iloc[12,0],3)
c_df.iloc[3,4]=round(clan.iloc[13,0],3)
c_df.iloc[3,5]=round(clan.iloc[14,0],3)
#age
c_df.iloc[6,0]=round(clan.iloc[6,0],3)
c_df.iloc[6,1]=round(clan.iloc[7,0],3)
c_df.iloc[6,2]=round(clan.iloc[8,0],3)
c_df.iloc[6,3]=round(clan.iloc[15,0],3)
c_df.iloc[6,4]=round(clan.iloc[16,0],3)
c_df.iloc[6,5]=round(clan.iloc[17,0],3)

#2nd row: CI
#income
c_df.iloc[1,0]=f"({round(clan.iloc[0,2],3)},{round(clan.iloc[0,3],3)})"
c_df.iloc[1,1]=f"({round(clan.iloc[1,2],3)},{round(clan.iloc[1,3],3)})"
c_df.iloc[1,2]=f"({round(clan.iloc[2,2],3)},{round(clan.iloc[2,3],3)})"
c_df.iloc[1,3]=f"({round(clan.iloc[9,2],3)},{round(clan.iloc[9,3],3)})"
c_df.iloc[1,4]=f"({round(clan.iloc[10,2],3)},{round(clan.iloc[10,3],3)})"
c_df.iloc[1,5]=f"({round(clan.iloc[11,2],3)},{round(clan.iloc[11,3],3)})"
#SES
c_df.iloc[4,0]=f"({round(clan.iloc[3,2],3)},{round(clan.iloc[3,3],3)})"
c_df.iloc[4,1]=f"({round(clan.iloc[4,2],3)},{round(clan.iloc[4,3],3)})"
c_df.iloc[4,2]=f"({round(clan.iloc[5,2],3)},{round(clan.iloc[5,3],3)})"
c_df.iloc[4,3]=f"({round(clan.iloc[12,2],3)},{round(clan.iloc[12,3],3)})"
c_df.iloc[4,4]=f"({round(clan.iloc[13,2],3)},{round(clan.iloc[13,3],3)})"
c_df.iloc[4,5]=f"({round(clan.iloc[14,2],3)},{round(clan.iloc[14,3],3)})"
#age
c_df.iloc[7,0]=f"({round(clan.iloc[6,2],3)},{round(clan.iloc[6,3],3)})"
c_df.iloc[7,1]=f"({round(clan.iloc[7,2],3)},{round(clan.iloc[7,3],3)})"
c_df.iloc[7,2]=f"({round(clan.iloc[8,2],3)},{round(clan.iloc[8,3],3)})"
c_df.iloc[7,3]=f"({round(clan.iloc[15,2],3)},{round(clan.iloc[15,3],3)})"
c_df.iloc[7,4]=f"({round(clan.iloc[16,2],3)},{round(clan.iloc[16,3],3)})"
c_df.iloc[7,5]=f"({round(clan.iloc[17,2],3)},{round(clan.iloc[17,3],3)})"

#3rd row: SE
#income
c_df.iloc[2,0]=f"-"
c_df.iloc[2,1]=f"-"
c_df.iloc[2,2]=f"[{round(clan.iloc[2,1],5)}]"
c_df.iloc[2,3]=f"-"
c_df.iloc[2,4]=f"-"
c_df.iloc[2,5]=f"[{round(clan.iloc[11,1],5)}]"
#SES
c_df.iloc[5,0]=f"-"
c_df.iloc[5,1]=f"-"
c_df.iloc[5,2]=f"[{round(clan.iloc[5,1],5)}]"
c_df.iloc[5,3]=f"-"
c_df.iloc[5,4]=f"-"
c_df.iloc[5,5]=f"[{round(clan.iloc[14,1],5)}]"
#age
c_df.iloc[8,0]=f"-"
c_df.iloc[8,1]=f"-"
c_df.iloc[8,2]=f"[{round(clan.iloc[8,1],5)}]"
c_df.iloc[8,3]=f"-"
c_df.iloc[8,4]=f"-"
c_df.iloc[8,5]=f"[{round(clan.iloc[17,1],5)}]"


c_df.rename(columns={0:"20% Most", 1:"20% Least", 2:"Difference", 3:"20% Most", 4:"20% Least", 5:"Difference"},index={0: "Income",1:"",2:"",3:"SES",4:"",5:"",6:"Age",7:"",8:""}, inplace=True)
#c_df.rename(columns={0:f"{group_means["method"][0]} 20% Most", 1:f"{group_means["method"][0]} 20% Least", 2:f"{group_means["method"][0]} Difference", 3:f"{group_means["method"][9]} 20% Most", 4:f"{group_means["method"][9]} 20% Least", 5:f"{group_means["method"][9]} Difference"},index={0: "Income",1:"",2:"",3:"SES",4:"",5:"",6:"Age",7:"",8:""}, inplace=True)
print(c_df)

c_df.to_csv("/content/drive/My Drive/ML/new2/Stage3_CLAN_satisfaction1.csv", index=False, header=True)

               20% Most        20% Least  ...        20% Least      Difference
Income            4.437            4.914  ...            4.826          -0.337
          (4.209,4.675)    (4.655,5.167)  ...    (4.606,5.067)  (-0.64,-0.014)
                      -                -  ...                -       [0.07831]
SES                0.95            0.962  ...            0.972          -0.009
          (0.938,0.963)     (0.95,0.973)  ...     (0.964,0.98)   (-0.02,0.003)
                      -                -  ...                -        [0.2337]
Age              14.203           12.491  ...           12.599           1.297
        (13.871,14.525)  (12.325,12.663)  ...  (12.378,12.816)    (0.88,1.695)
                      -                -  ...                -           [0.0]

[9 rows x 6 columns]
